In [90]:
import os
import tqdm
import glob
from sklearn.model_selection import train_test_split
from struct import unpack
import shutil

In [45]:
%cd ../../data/image-database

/home/tomas/Development/azurecup-2021-triumviratus/data/image-database


In [46]:
!mkdir maysee-mushrooms-classification-common-genuss-images
%cd maysee-mushrooms-classification-common-genuss-images

/home/tomas/Development/azurecup-2021-triumviratus/data/image-database/maysee-mushrooms-classification-common-genuss-images


# Download the dataset

In [47]:
os.environ['KAGGLE_CONFIG_DIR'] = '../../../backend/keychain/kaggle/' 

In [48]:
!kaggle datasets download -d maysee/mushrooms-classification-common-genuss-images

100%|██████████████████████████████████████| 1.85G/1.85G [09:28<00:00, 5.18MB/s]
100%|██████████████████████████████████████| 1.85G/1.85G [09:28<00:00, 3.50MB/s]


# Unzip data

In [49]:
import zipfile

In [80]:
local_zip = './mushrooms-classification-common-genuss-images.zip'

with zipfile.ZipFile(local_zip,'r') as zip_ref:
    zip_ref.extractall('')

In [72]:
# !rm mushrooms-classification-common-genuss-images.zip
!rm -rf mushrooms

# Remove broken files

In [83]:
marker_mapping = {
    0xffd8 : "Start of Image",
    0xffe0 : "Application Default Header",
    0xffdb : "Quantization Table",
    0xffc0 : "Start of Frame",
    0xffc4 : "Define Huffman Table",
    0xffda : "Start of Scan",
    0xffd9 : "End of Image"
}

class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()
    
    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])

            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2 + lenchunk:]            
            if len(data) == 0:
                break        


In [85]:
bads = []

for img in tqdm.tqdm(glob.glob('Mushrooms/**/*')):
  image = JPEG(img)
  try:
    image.decode()   
  except:
    bads.append(img)

for name in tqdm.tqdm(bads):
  print('[X]',name)
  os.remove(name)


100%|██████████| 6714/6714 [00:08<00:00, 754.22it/s]

100%|██████████| 3/3 [00:00<00:00, 985.97it/s]

[X] Mushrooms/Russula/092_43B354vYxm8.jpg
[X] Mushrooms/Russula/138_QBvBBlSQ-Zs.jpg
[X] Mushrooms/Russula/139_H3FH6JjjTzg.jpg


In [92]:
%mkdir train
%mkdir test

mkdir: cannot create directory ‘train’: File exists
mkdir: cannot create directory ‘test’: File exists


In [ ]:
data = glob.glob('Mushrooms/**/*')

In [93]:
names = set()

for file in tqdm.tqdm(data):
  name = file.replace('Mushrooms/','').split('/')[0]
  names.add(name)

for name in tqdm.tqdm(names):
  for set_name in ('train','test'):
    os.makedirs('{}/{}'.format(set_name, name), exist_ok = True)



100%|██████████| 6711/6711 [00:00<00:00, 535173.29it/s]

100%|██████████| 9/9 [00:00<00:00, 15178.42it/s]


In [94]:
train, test = train_test_split(data, test_size = 0.2)

for source in tqdm.tqdm(train):
  target = source.replace('Mushrooms', 'train')
  shutil.copy(source,target)

for source in tqdm.tqdm(test):
  target = source.replace('Mushrooms', 'test')
  shutil.copy(source,target)



100%|██████████| 5368/5368 [00:05<00:00, 1062.11it/s][A

100%|██████████| 1343/1343 [00:07<00:00, 184.01it/s][A
